# Advanced Python

In [ ]:
import base64
from IPython.display import Image, display

def mermaid(graph):
    graphbytes = graph.encode("ascii")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(
        Image(
            url="https://mermaid.ink/img/"
            + base64_string
        )
    )

In [ ]:
print("Hello, World!")

# Module 01 - Files

## Lab 01.01 - Files and Lists

In [ ]:
with open("names.txt", encoding= "utf8") as file:
    names: list[str] = [line.rstrip("\n") for line in file.readlines()]

longest_name = max(names, key = len)
shortest_name = min(names, key = len)

print(f"Longest name is: {longest_name}")
print(f"Shortest name is: {shortest_name}")


In [ ]:
del names
del shortest_name
del longest_name
del file

## Lab 01.02 - Sorting Files

In [ ]:
import shutil

shutil.copyfile("students.txt", "students.bak")

In [ ]:
FILENAME = "students.txt"
with open(FILENAME, encoding = "utf8") as infile:
    names = infile.readlines()

print(f"Original order: {names}")

with open(FILENAME, mode = "w", encoding = "utf8") as outfile:
    sorted_names = sorted(names)
    outfile.writelines(sorted_names)

print(f"Sorted Order: {sorted_names}")

In [ ]:
import shutil
import os

shutil.copyfile("students.bak", "students.txt")
os.remove("students.bak")

## Lab 01.03 - File Exception Handling
### Part 1 - Random
1. Collect user input on the name of the file to be opened
1. If the file exists, read a random line from the file
1. Otherwise, prompt the nuser to give another file name until they give you a file that exists

In [ ]:
from random import choice

#filename = input("Enter the file to be opened: ")
filename = "students.txt"

file_exists = False
while not file_exists:
    try:
        infile = open(filename, "r")
        file_exists = True
    except FileNotFoundError:
        filename = input("Please enter a file that ACTUALLY exists: ")

lines = [line.rstrip("\n") for line in infile.readlines()]
infile.close()
print(f"Here's a random line from {filename}: {choice(lines)}")

In [ ]:
del choice
del filename
del file_exists
del infile
del lines

# Module 2 - Relational Databases
* Relational databases separate data by table
* CREATE TABLE
* INSERT
* SELECT
* UPDATE
* DELETE

## Connection and Cursor
* SQLite built into Python 3
* Before working with a database, you must connect to it
* A `Cursor` is used to execute the SQL command strings
* The `execute()` function takes a string argument

In [ ]:
import sqlite3
from os import path, remove

DATABASE = "demo.db"
if path.exists(DATABASE):
    remove(DATABASE)

conn = sqlite3.connect(DATABASE)
cursor = conn.cursor()

## Executing Commands
* `CREATE TABLE`

  Possible data types:
  * `INTEGER` - stores integer numbers
  * `TEXT` - stores string values
  * `REAL` - stores floating point numbers

In [ ]:
command = """
CREATE TABLE table_name (column_1 TEXT, column_2 INTEGER)
"""
cursor.execute(command)
print("A new table with two columns has been created")


* `INSERT INTO`
  ```sql
  command = INSERT INTO table_name VALUES ("Value 1", 12345), ("Value 2", 6789)
  ```
  ```python
  cursor.execute(command)
  ```
  * Database will look like this

    | column_1 | column_2 |
    |:---------|:---------|
    | Value 1  | 1234     |
    | Value 2  | 5678     |

  * **Note:** Data will not persist by default it must be committed with `connection.commit()`

In [ ]:
command = """
INSERT INTO table_name VALUES
    ("Value 1", 1234),
    ("Value 2", 5678)
"""
cursor.execute(command)
conn.commit()
print("The entries have been added")


* `SELECT FROM`
  * Selects data from the columns
  * **Note:** `fetchall()` pulls the data from the cursor and returns the rows in a list of tuples

In [ ]:
command = """
SELECT * FROM table_name
"""
cursor.execute(command).fetchall()


* `WHERE`
  * Operators

    | Symbol | Meaning               |
    |:-------|:----------------------|
    | `=`    | equals                |
    | `!=`   | not equals            |
    | `>`    | greater than          |
    | `<`    | less than             |
    | `>=`   | greater than or equal |
    | `<=`   | less than or equal    |

* Dynamic Statements
  * The question mark can be used to indicate a place for dynamic option input in an SQL statement
  

In [ ]:
command = """
SELECT * FROM table_name
WHERE column_2 > ?
"""
cursor.execute(command, (2000,)).fetchall()

In [ ]:
del command
del DATABASE
cursor.close()
conn.close()
del cursor
del conn

## Lab 02.01 - Corrupted Data
1. Bring the rock_store_backup.db, new_equipment.csv, updated_prices.csv into your environment
1. THe company recently started selling mining equipment and wanted to add the new stock to the database
   * Examine the contents of new_equipment.csv
   * Create a table called equipment with areas for all the data in `new_equpment.csv`
1. Next open the file and loop through the data, inserting it into the `equpment` table
1. Next use `updated_prices.csv` to update the `rocks` table (already in the database) entries' price values in the csv
1. Next delete all the entries in the rocks table that have a stock of 0. They will not be restocked.

_Optional Challenge:_
1. Create a function that prints formatted outputs from the database tables
1. Create a single function that can be called to print for any table/database

In [ ]:
from shutil import copyfile

copyfile("rock_store_backup.db", "rock_store.db")
del copyfile

In [ ]:
from functools import reduce
import sqlite3
from contextlib import closing

def read_csv(filename: str) -> list[tuple]:
    with open(filename, encoding = "utf8") as equipment:
        return [tuple(line.rstrip("\n").split(",")) for line in equipment.readlines()]
    
def print_table(cursor: sqlite3.Cursor, table: str):
    rows: list[tuple] = cursor.execute(f"SELECT * FROM {table}").fetchall()
    
    number_columns = len(rows[0])
    column_lengths: list[int] = []
    for column in range(number_columns):
        lengths = [len(str(row[column])) for row in rows]
        column_lengths.append(max(lengths))

    for row in rows:
        print("|", end="")
        for column in range(number_columns):
            print(f" {row[column]:{column_lengths[column]}}", end= " |")
        print()
        

with closing(sqlite3.connect("rock_store.db")) as connection:
    with closing(connection.cursor()) as cursor:
        create_new_table = """
        CREATE TABLE equipment
        (
            id INTEGER,
            name TEXT,
            category TEXT,
            quantity INTEGER,
            price REAL
        )
        """
        cursor.execute(create_new_table)

        items = read_csv("new_equipment.csv")
        
        # Insert the items
        for item in items:
            cursor.execute("INSERT INTO equipment VALUES(?, ?, ?, ?, ?)", item)
        connection.commit()

        # Output the entries
        print("This is the content of the new equipment table")
        print_table(cursor, "equipment")

        # Update the prices
        updated_rocks = read_csv("updated_prices.csv")

        for rock in updated_rocks:
            update_command = """
                             UPDATE rocks SET price = ?
                             WHERE id = ?
                             """
            cursor.execute(update_command, (rock[3], rock[0]))
        connection.commit()

        # Display the new rock list
        print("\nThis is the content of the rocks table")
        print_table(cursor, "rocks")

        # Get your rocks off (the table)
        no_stock = cursor.execute("SELECT (id) FROM rocks WHERE stock = 0").fetchall()

        for rock in no_stock:
            cursor.execute("DELETE FROM rocks WHERE id = ?", rock)

        # Print rocks again
        print("\nUnstocked rocks were removed. Here is the updated rocks table:")
        print_table(cursor, "rocks")

In [ ]:
del connection
del create_new_table
del cursor
del item
del items
del no_stock
del rock
del update_command
del updated_rocks

# Module 3 - Object Oriented Programming

## Lab 03.03
1. Modify `Student` and include two attributes
   * a private (`__`) attribute "name"
   * a protected (`_`) attribute "grade"
   * `grade` represents their current year or school
1. Practice accessing and mutating the variables with and without the getter/setter methods

_Optional Challenge:_
1. Add a `__str__` method that describes the state of a student object
1. Cause an error if the grade value is less than 1 or greater than 12
   * Be sure it works on new instances and when changing the grade of a current instance

In [ ]:
import sys

class Student:
    __name: str
    _grade: int

    def __init__(self, name: str, grade: int):
        self.__name = name
        self.grade = grade

    @property
    def name(self) -> str:
        return self.__name
    
    @property
    def grade(self) -> int:
        return self._grade
    
    @grade.setter
    def grade(self, value) -> None:
        if value >= 1 and value <= 12:
            self._grade = value
        else:
            raise ValueError("Grade cannot be below 1 or above 12")
    
    def attend_class(self) -> None:
        pass

student = Student("Nicholas", 5)

print("Since Student's name has two underscores, attempting to access it will get an attribute error")
print(f"If the access succeeds we will see 'The student's name is {student._Student__name}'")
print("Otherwise we will catch the exception.")
try:
    print(f"The student's name is {student.__name}")
except AttributeError:
    print("We got an AttributeError!")
print()

print("Now we will access Student's grade without a getter")
print(f"The student's grade is {student._grade}")
print()

print("Now let's try to set the properties!")
print("'name' does not have a property setter, so we will get an exception")
try:
    student.name = "New Name"
    print(f"The student's new name is {student.name}")
except AttributeError:
    print("We got an AttributeError")

print()
print("Now let's set 'grade'!")

old_grade = student.grade
student.grade = 8

print(f"Student's grade was changed from {old_grade} to {student.grade}")
print()

print("Now let's set 'grade' to something out of bounds")
try:
    student.grade = 100
except ValueError:
    print("We got a ValueError!")
    print(f"The exception says '{sys.exception()}'")

In [ ]:
del old_grade
del student

## Inheritance

### Overloading in Python
* Overloading doesn't work in Python
* Causes an error when the first method is called
* **Note:** Interpreter recognizes the **LAST ONE POSITIONALLY**

In [ ]:
import sys

class SpaceShip:
    def go(self, speed):
        print(f"Go with only speed {speed}")

    def go(self, speed, warp_faster):
        print(f"Go with speed and warp factor {speed}, {warp_faster}")

SpaceShip().go(1, True)
try:
    SpaceShip().go(1)
except:
    print(sys.exception())

## Lab 03.04 - Student Types
1. Create two subclasses of `Student`
   * `HighSchoolStudent`
   * `CollegeStudent`
1. Both classes need an `attend_class` method
   * For high school, print a message that attendance is required
   * For college, print a message that attendance is optional
1. Both classes should override `__str__`
1. The `grade` attribute can now be as high as 16
1. Create a list of at least 6 students in various grades, loop through it and call their methods

_Optional Challenge:_
1. College students can do research. Use multiple inheritance to solve this. Also, how can we handle it in our loop from step 5?
1. Populate the list of students programmatically

In [ ]:
from abc import ABCMeta, abstractmethod

class Researcher(metaclass = ABCMeta):
    @abstractmethod
    def do_reasearch(self) -> None:
        pass

In [ ]:
class HighSchoolStudent(Student):
    def attend_class(self):
        print("Attendance is mandatory")

    def __str__(self) -> str:
        return f"HighSchoolStudent(name: {self.name}, grade: {self.grade})"

class CollegeStudent(Student, Researcher):
    @property
    def grade(self) -> int:
        return self._grade

    @grade.setter
    def grade(self, value) -> None:
        if value >= 1 and value <= 16:
            self._grade = value
        else:
            raise ValueError("College grades can only be between 1 and 16")

    def attend_class(self):
        print("Attendance is optional")

    def do_reasearch(self) -> None:
        print("Searching for the meaning of life...")

    def __str__(self) -> str:
        return f"CollegeStudent(name: {self.name}, grade: {self.grade})"

In [ ]:
from faker import Faker
from random import choice, randint, seed

def get_random_students(number: int) -> list[Student]:
    fake = Faker()
    fake.seed_instance(0)
    seed(0)

    STUDENT_FACTORY: list[function] = [
        lambda: CollegeStudent(fake.first_name(), randint(13, 16)),
        lambda: HighSchoolStudent(fake.first_name(), randint(1, 12))
        ]

    return [choice(STUDENT_FACTORY)() for _ in range(number)]

for student in get_random_students(6):
    print(student)
    student.attend_class()
    
    if isinstance(student, Researcher):
        student.do_reasearch()
    print()

In [ ]:
del student
del choice
del randint
del seed

## Lab 03.05 - StudentAbc
1. Convert the Student class from the previous lab to an `Abstract Base Class`
1. What method(s) should be abstract?
1. How does this change affect the design and extensibility of your code?

_Optional Challenge:_
1. Create an abstract property `age` in the `StudentAbc` and demonstrate how to call it

In [ ]:
from abc import ABC, abstractmethod

class StudentAbc(ABC):
    __name: str
    __grade: int

    def __init__(self, name: str, grade: int) -> None:
        self.__name = name
        self.grade = grade

    @property
    def name(self) -> str:
        return self.__name
    
    @property
    def grade(self) -> int:
        return self.__grade
    
    @grade.setter
    def grade(self, value) -> None:
        self.__grade = value

    @property
    @abstractmethod
    def age(self) -> int:
        pass

    @age.setter
    @abstractmethod
    def age(self, value) -> None:
        pass

    @abstractmethod
    def attend_class(self):
        pass

In [ ]:
class CollegeStudent(StudentAbc):
    __age: int

    def __init__(self, name: str, grade: int, age: int) -> None:
        super().__init__(name, grade)
        self.age = age

    @property
    def age(self):
        return self.__age
    
    @age.setter
    def age(self, value):
        if value >= 18:
            self.__age = value
        else:
            raise ValueError("College Students must be at least 18")
    
    def attend_class(self):
        print("Attendance is optional")

college_student = CollegeStudent("Nicholas", grade=5, age=27)

try:
    print("The subclass does not allow ages below 18. We're going to try to set the age to 5")
    college_student.age = 5
except ValueError:
    print("We got a a value error when we tried that")

In [ ]:
del college_student

# Module 4 - Python Features

## Lab 04.01 - Docstring Practice
1. Create a simple function that takes two integers, multiplies them, and returns the result
1. Practice documenting the function with docstrings

In [ ]:
def multiply(a: int, b: int) -> int:
    """
    Accepts two integers and multiplies them together.

    Returns the result
    """
    return a * b

In [ ]:
import pydoc

with open("out.html", "w") as out:
    out.write(pydoc.html.document(multiply))

In [ ]:
import os

del out
os.remove("out.html")

## Lab 04.02 - Monkey Patching Exploration
1. Create a class called IRS
1. Create a method `calculate_tax(self, income)`
1. The functionallity is not ready yet, just leave the method empty
1. Create a `Client` class with a `calculate_my_income_tax(self, income)` method
1. This class uses a reference of IRS to call `calculate_tax`
1. Create a `mock_calculate_tax(self, income)` method that just returns a flat 20% tax on any income
1. Use monkey patching to ensure the right method is invoked

_Optional Challenge:_
1. Try monkey patching a module

In [ ]:
class IRS:
    def calculate_tax(self, income: float) -> float:
        pass

class Client:
    __irs: IRS

    def __init__(self) -> None:
        self.__irs = IRS()

    def calculate_my_income_tax(self, income: float):
        return self.__irs.calculate_tax(income)

def mock_calculate_tax(self, income: float) -> float:
    return income * .20

def monkey_patch() -> None:
    IRS.calculate_tax = mock_calculate_tax

client: Client = Client()
monkey_patch()
print(f"After monkey patching, the income tax amount is {client.calculate_my_income_tax(100)}")

In [ ]:
import random

print("Now we're going to always return the number 5 from 'randint'")
random.randint = lambda a, b: 5
print(f"Here is a list of 10 'random' integers {[random.randint(0, 100) for _ in range(10)]}")


In [ ]:
del client

## Lab 04.03 - Fibonacci Sequence
1. Using a generator function, yield the integers in the Fibonacci Sequence up to the specified limit
1. Obtain user input on what the limit should be

In [ ]:
def fib(limit: int):
    last_fib = 0
    current_fib = 1

    if limit > last_fib:
        yield last_fib

    while(current_fib < limit):
        yield current_fib

        last_fib += current_fib
        last_fib, current_fib = current_fib, last_fib

def fib_comprehension(limit: int):
    root_5 = 5 ** 0.5
    phi = (1 + root_5) / 2
    return (round(phi ** i / root_5) for i in range(limit))
list(fib_comprehension(10))

In [ ]:
number_str = ""

while not str.isdecimal(number_str):
    # number_str = input("Input a number between 1 and 100: ")
    number_str = "5"

number = int(number_str)

squares = (f"{str(n * n)},\n" for n in range(1, number + 1))

with open("squares.txt", "w") as outfile:
    for square in squares:
        outfile.write(square)

In [ ]:
import os

del number_str
del number
del square
del squares
del outfile

os.remove("squares.txt")

# Module 5 - Unit Testing

## Lab 05.01 - Testing String Methods
### Part 1 - Write the tests
1. Create a testing class that subclasses `unittest.TestCase`
1. Choose three string methods that are different from the example
1. Write a test method for each string method

_Optional Challenge_
1. Use `assertRaises()` with the `with` statement in one of your test methods

### Part 2 - Run the tests
1. Use the following block to run your tests
   ```python
   if __name__ == '__main__':
       unittest.main()
   ```
2. Change one of your test methods to make it intentionally fail

In [ ]:
import unittest

class TestStrings(unittest.TestCase):
    def test_isdecimal(self):
        self.assertTrue("12345".isdecimal())
        self.assertFalse("abcd".isdecimal())

def test():
    unittest.main(argv=[''], verbosity=2, exit=False)

test()

In [ ]:
del TestStrings

## Lab 05.02 - Animal Shelter
### Part 1 - Creating Your Function
1. Create an `AnimalShelter` class
1. Create a function inside the `AnimalShelter` class called `add_method()`
1. Your function should take a list of animals to add to the animal shelter
1. Assume that our animal shelter can only hold 10 animals
1. Raise a `ValueError` if the list contains more than 10 animals
1. Return the list of animals that have been added to the animal shelter

### Part 2 - Testing Your Function
1. Create an `AnimalShelterTest` class should subclass `unittest.TestCase`
1. Create two test methods, `test_add_animals_success()` and `test_add_animals_exception()`
1. The first test method should cover any list that contains 10 animals or less
1. The second test method method should cover lists that contain more than 10 animals

In [ ]:
class AnimalShelter:
    __animals: list[str]

    def __init__(self):
        self.__animals = []

    def add_animals(self, animals: list[str]):
        if len(animals) + len(self.__animals) > 10:
            raise ValueError("The shelter can only hold 10 animalsS")
        
        self.__animals += animals

        return animals

In [ ]:
import unittest

class AnimalShelterTest(unittest.TestCase):
    def setUp(self) -> None:
        self.shelter = AnimalShelter()

    def test_add_animals_success(self):
        animals = ["Lion", "Tiger", "Bear"]
        shelter = AnimalShelter()
        self.assertListEqual(shelter.add_animals(animals), animals)

        animals = ["Cat", "Dog"]
        self.assertListEqual(shelter.add_animals(animals), animals)

        animals = ["Penguin", "Giraffe", "Zebra", "Hippo", "Mouse"]
        self.assertListEqual(shelter.add_animals(animals), animals)

    def test_add_animals_exception(self):
        animals = ["Lion",    "Tiger",   "Bear",  "Cat",   "Dog",
                   "Penguin", "Giraffe", "Zebra", "Hippo", "Mouse"]
        shelter = AnimalShelter()
        shelter.add_animals(animals)

        with self.assertRaises(ValueError):
            shelter.add_animals(["Straw"])

test()

In [ ]:
del AnimalShelterTest

# Module 06 - Debugging and Logging

## Lab 06.01 - Debugging
1. Debug the provided the below code block
1. As you'll see from the code - one should have 4 members and the other band should have 5
1. When we run the code both bands have 5

In [ ]:
imagine_dragons = ['dan reynolds', 'ben mcKee', 'wayne sermon', 'daniel platzman']

stone_dragons = imagine_dragons
stone_dragons.append("mic jagger")

def display_members(band):
    print("\nThe band members are: ")
    for item in enumerate(band):
        for s in item:
            print(str(s).title())

# wait, how did Mic get in two bands?
if __name__ == '__main__':
    display_members(imagine_dragons)
    display_members(stone_dragons)

In [ ]:
del imagine_dragons
del stone_dragons

1. Debug the below code block
1. We want you to compare two ranges

In [ ]:
# We want to compare two ranges of numbers in a nested loop 
# We want to note when the values don't match
# We have a bug.  See for example, this output:  They do NOT match 259 != 259

i = 240
while i < 260:
    for y in range(240,260):
        if i is y:
            print(f'They match {i} = {y}')
        else:
            print(f'They do NOT match {i} != {y}')
    i += 1

In [ ]:
del i
del y

## Lab 06.03 - Basic Logging

1. Create a script that does the following  
   1. Sets up a `basicConfig` with an output file named `basic.log`
   1. Has a function that determines if a number is even or odd
   1. Writes to a log every time the function is called if the logging level is debug
   1. Writes if the number is even or odd when the logging level is info
   1. Catches and writes an error to the log if a non-numeric character is passed to the function
1. Change the logging level to `DEBUG`, `INFO`, and `ERROR` to see the different outputs in your log file
   

In [ ]:
import logging

logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().handlers.clear()
logging.getLogger().handlers.append(logging.FileHandler("basic.log", mode="w"))

def even_or_odd(value: int | float) -> str:
    logging.debug(f"{even_or_odd.__name__} has been called")
    
    try:
        return_str = "even" if value % 2 == 0 else "odd"
        logging.info(f"The number {value} is {return_str}")
        return return_str
    except TypeError:
        logging.error("The user didn't pass a number to this function")
        
even_or_odd(2)
even_or_odd(5)
even_or_odd("5")

In [ ]:
import os
import logging

logging.getLogger().handlers.clear()
os.remove("basic.log")

## Lab 06.04 - Custom Logging

In [ ]:
import logging

class BankLogger:
    __logger: logging.Logger
    
    def __init__(self) -> None:
        self.__logger = logging.getLogger("TransactionLog")
        self.logger.setLevel(logging.DEBUG)
        self.logger.handlers.clear()
        
        handler = logging.FileHandler(f"{self.logger.name}.log", mode="w")
        handler.setLevel(logging.DEBUG)
        
        formatter = logging.Formatter("%(asctime)s: %(levelname)s - %(message)s")
        handler.setFormatter(formatter)
        
        self.logger.addHandler(handler)
        
    @property
    def logger(self) -> logging.Logger:
        return self.__logger

In [ ]:
class BankAccount:
    __balance: float
    __logger: BankLogger
    
    def __init__(self) -> None:
        self.__balance = 0
        self.__logger = BankLogger()
    
    def deposit(self, amount: float) -> None:
        self.__balance += amount
        self.__logger.logger.info("A deposit of %g was made: resulting in a balance of %g", amount, self.__balance)
        
    def withdraw(self, amount: float) -> bool:
        if self.__balance - amount < 0:
            self.__logger.logger.error("The account attempted to withdraw %g from a balance of %g", amount, self.__balance)
            return False
        
        self.__balance -= amount

In [ ]:
account = BankAccount()

account.deposit(5)
account.withdraw(100)

In [ ]:
import logging
import os

logging.getLogger("TransactionLog").handlers.clear()
os.remove("TransactionLog.log")

del account

# Module 07 - Design Patterns

## Lab 07.01 - Factory Pattern
1. We have two kinds of products (so far): virtual courses and on-site courses
1. We have to create materials in both cases, but the way the materials are created varies
1. Create the appropriate classes to handle this variation
   * Print a simple message when a method is called
1. Create a `CourseFactory` that hides the product types from clients and calls the correct `create_materials` method

In [ ]:
import json

with open("factory.json", mode= "w") as config:
    json.dump({"course_type": "virtual"}, config)

In [ ]:
from abc import ABC, abstractmethod
from json import load
from typing import Literal

class Course(ABC):
    @abstractmethod
    def get_course_description(self):
        pass

class VirtualCourse(Course):
    def get_course_description(self) -> str:
        return "A class taken virtually"

class OnSiteCourse(Course):
    def get_course_description(self):
        return "A class taken on-site"

class CourseFactory(ABC):
    @abstractmethod
    def create_course(self) -> Course:
        pass

    def get_course_description(self) -> str:
        course = self.create_course()

        return course.get_course_description()

class VirtualCourseFactory(CourseFactory):
    def create_course(self) -> Course:
        return VirtualCourse()

class OnSiteCourseFactory(CourseFactory):
    def create_course(self) -> Course:
        return OnSiteCourse()

def get_factory(type: Literal["virtual"] | Literal["onsite"]) -> CourseFactory:
    match type.lower():
        case "virtual":
            return VirtualCourseFactory()
        case _:
            return OnSiteCourseFactory()

def load_factory_from_json(filename: str) -> CourseFactory:
    with open(filename) as config:
        contents = load(config)

        factory_name: str = contents["course_type"]
        return get_factory(factory_name)

In [ ]:
def get_some_description(factory: CourseFactory):
    print(factory.get_course_description())

factory = load_factory_from_json("factory.json")

get_some_description(factory)
get_some_description(get_factory("onsite"))

In [ ]:
import os

os.remove("factory.json")

del factory

## Lab 07.02 - Adapter Problem
1. Use the supplied `BankAccount` class below
1. We have acquired a new bank! Unfortunately their `withdrawal` method is named `make_a_withdrawal`.
1. Create the appropriate _target_ and _adapter_ classes to allow the new bank to call our `BankAccount` withdrawal method

In [ ]:
class BankAccount:
    def __init__(self, initial_deposit=0):
        self._balance = initial_deposit

    @property
    def balance(self):
        return self._balance

    @balance.setter
    def balance(self, value):
        self._balance = value

    def deposit(self, deposit_amt):
        self.balance += deposit_amt
        dep_msg = f"A deposit of {deposit_amt} was added to the account.  The new balance is {self.balance}"
        print(dep_msg)

    def withdrawal(self, withdraw_amt):
        wd_msg = f"Withdraw amount of {withdraw_amt} attempted when account balance was {self.balance}"
        if withdraw_amt < self.balance:
            self.balance -= withdraw_amt
            wd_msg = f"Withdrawal of {withdraw_amt} recorded.  New balance is {self.balance}"
            print(wd_msg)
        else:
            raise ValueError("Withdrawl amount cannot exceed balance")

In [ ]:
from abc import ABC, abstractmethod

class BankAccountTarget(ABC):
    @property
    @abstractmethod
    def balance(self):
        pass

    @balance.setter
    @abstractmethod
    def balance(self, value):
        pass

    @abstractmethod
    def make_a_withdrawal(self, withdrawal_amount: float):
        pass

    @abstractmethod
    def deposit(self, deposit_amount: float):
        pass

class BankAccountAdapter(BankAccountTarget):
    __adaptee: BankAccount

    def __init__(self, account: BankAccount) -> None:
        self.__adaptee = account

    @property
    def balance(self):
        return self.__adaptee.balance

    @balance.setter
    def balance(self, value):
        self.__adaptee.balance = value

    def make_a_withdrawal(self, withdrawal_amount: float):
        self.__adaptee.withdrawal(withdrawal_amount)

    def deposit(self, deposit_amount: float):
        self.__adaptee.deposit(deposit_amount)

In [ ]:
acctOne = BankAccount()
acctOne.deposit(10.00)
acctOne.deposit(90.00)
acctOne.withdrawal(50.00)
print(f"Normal Account's final balance is {acctOne.balance}")

acctTwo = BankAccountAdapter(BankAccount())
acctTwo.deposit(10.00)
acctTwo.deposit(90.00)
acctTwo.make_a_withdrawal(50.00)
print(f"Account Adapter's final balance is {acctTwo.balance}")

## Lab 07.03 - Proxy
1. Create an `ABC` with an abstract method `transaction`
1. Write a `CreditCard` class that extends the `ABC`
1. Override the `transaction` method and have it print a message reflecting the current balance each time it's called
1. Create a _proxy_ class that extends your `ABC`
1. Override the `transaction` method and have it do 3 things
   1. Write a **log message** that a card transaction in the amount of ___ was received
   1. Call the actual transaction method in `CreditCard`
   1. Write a **log message** that a transaction in the amount of ___ was completed

In [ ]:
from abc import ABC, abstractmethod

class Transactor(ABC):
    @abstractmethod
    def transaction(self, amount: float):
        pass

class CreditCard(Transactor):
    __balance: float

    def __init__(self) -> None:
        self.__balance = 0

    def transaction(self, amount: float):
        print(f"A transaction of {amount} has occurred resulting in a balance of {self.__balance}")
        self.__balance -= amount

In [ ]:
import logging

class TransactorProxy(Transactor):
    __transactor: Transactor
    __logger: logging.Logger
    
    def __init__(self, transactor: Transactor) -> None:
        self.__transactor = transactor
        self.__logger = logging.getLogger()
        
        self.__logger.handlers.clear()
        
        handler = logging.FileHandler(f"Transactions.log", mode="w")
        handler.setLevel(logging.DEBUG)
        
        formatter = logging.Formatter("%(levelname)s: - %(message)s")
        handler.setFormatter(formatter)
        
        self.__logger.addHandler(handler)
        
    def transaction(self, amount: float):
        self.__logger.info("A transaction of $%g was received", amount)
        self.__transactor.transaction(amount)
        self.__logger.info("A transaction of $%g was completed", amount)

In [ ]:
proxy = TransactorProxy(CreditCard())

proxy.transaction(50.00)
proxy.transaction(1000.00)

In [ ]:
import os
import logging

logging.getLogger().handlers.clear()
os.remove("Transactions.log")

del proxy

## Lab 07.04 - Strategy
1. Create an `Order` class that takes an amount when created
1. Add a method that calculated the tax on the order amount
1. An order must always be taxed but the way it's taxed varies
   * US Tax is 5%
   * Canadian Tax if 8%
   * EU Tax is 11%
1. Create at least 3 orders to demonstrate you can calculate the taxes correctly
1. What would happen if
   1. We needed to tax orders in Japan at 10%
   1. Senior citizens in Canada don't pay tax at all
   1. Each US State had a different tax rate

In [ ]:
from abc import ABC, abstractmethod

class TaxStrategy(ABC):
    @abstractmethod
    def calculate_tax(self, amount: float) -> float:
        pass

class Order:
    __amount: float
    __strategy: TaxStrategy
    
    def __init__(self, amount: float, strategy: TaxStrategy) -> None:
        self.__amount = amount
        self.__strategy = strategy
        
    def calculate_tax(self) -> float:
        return self.__strategy.calculate_tax(self.__amount)

In [ ]:
class USTax(TaxStrategy):
    def calculate_tax(self, amount: float) -> float:
        return amount * 0.05

class CanadaTax(TaxStrategy):
    def calculate_tax(self, amount: float) -> float:
        return amount * 0.08

class EUTax(TaxStrategy):
    def calculate_tax(self, amount: float) -> float:
        return amount * 0.11

class JapanTax(TaxStrategy):
    def calculate_tax(self, amount: float) -> float:
        return amount * 0.10

In [ ]:
order = Order(20.00, USTax())
print(f"A user in the US made an order of $20.00 which resulted in a tax of {order.calculate_tax()}")

canadian_order = Order(48.99, CanadaTax())
print(f"A Canadian user made an order of $48.99 which resulted in a tax of {canadian_order.calculate_tax()}")

eu_order = Order(15_569.78, EUTax())
print(f"A user in the EU made an order of €15,569.78 which resulted in a tax of {eu_order.calculate_tax()}")

# Module 08 - RESTful Web Services and Clients

## Lab 08.01 - Environment Setup
1. Install `requests`
1. Install `Flask`
1. Use the code below to test the installation

In [ ]:
import requests

url = "https://httpbin.org/status/200"
response = requests.get(url)
print(response.status_code)

## Lab 08.02 - Making Requests
1. In a separate terminal run the module
1. The `testing_api.py` module contains a mock REST web API for top rated video games
1. Access the api using the url `http://localhost/top_games` and `http://localhost/top_games/<id>`

In [ ]:
import requests

# Get the list of games
url = "http://localhost/top_games"
response = requests.get(url)
print("This is the list of the top games:")
print(response.json(), end="\n\n")
backup_games: dict = response.json()

game = {
        "description": "A misunderstood turtle must deal with the abusive ex-boyfirend of his girlfriend.",
        "title": "Super Mario Brothers",
        "released": 1986,
        "rating": 8.5
        }
requests.post(url, json=game)

response = requests.get(url)
print("A new game has been added to the list:")
print(response.json(), end="\n\n")

bad_game = {
    "title": "Bad Ratz",
    "description": "THE BEST GAME EVER!",
    "released": 2007,
    "rating": 0.0
}

requests.put(f"{url}/1", json=bad_game)
response = requests.get(url)
bad_games = response.json()

print("A malicious hacker has added a different game to the number 1 spot!")
print(response.json(), end="\n\n")

print("Luckily the website had a backup they could restore from")

for game_key in backup_games:
    requests.put(f"{url}/{game_key}", json=backup_games[game_key])

for game_key in range(len(backup_games) + 1, len(bad_games) + 1):
    requests.delete(f"{url}/{game_key}")

response = requests.get(url)
print(response.json())
print("Phew! That was close!")

# Module 09 - Python Optimization

## Lab 09.01 - Binary Search vs. Linear Search
### Part 1 - Binary Search
1. Binary search is a searching algorithm for sorted collections of integers
1. Create a function called `binary_search()`
1. THis function should take both a list of integers and the target integer
1. It should return the index at which the target integer is found.
1. Return -1 if the integer is not found

### Part 2 - Linear Search
1. Create a function called `linear_search()`
1. This function should take the same parameters as `binary_search()`
1. Use the `timeit` module to find the amount of time these functions take to complete their task.

In [ ]:
def binary_search(lst: list[int], target: int):
    length = len(lst)
    half_index = length // 2
    half = lst[half_index]

    while half != target:
        half_index = half_index // 2 if half > target else half_index + (length - half_index) // 2
        half = lst[half_index]

    return half_index

def linear_search(lst: list[int], target: int):
    for i in range(lst):
        if lst[i] == target:
            return i

    return -1

In [ ]:
import timeit
import random

lst = list(range(100))
target = random.randint(0, 99)

# timeit.timeit(lambda: binary_search(lst, target), number=50)

## Lab 09.02 - Recursion
1. Create a recursive function that calculates factorials
1. The function should take an integer and return the result of the factorial
1. Use the `cProfile.run()` function to collect statistics on the performance of your recursive function

_Optional Challenge:_
1. Use the `pstats` module to gather information about how many primitive and recursive calls occur

In [ ]:
import cProfile

def factorial(n: int):
    if n == 1:
        return 1
    elif n <= 0:
        raise ValueError("Factorial is undefined for numbers 0 or less")

    return factorial_tail(n - 1, n)

def factorial_tail(n: int, acc: int):
    if n == 1:
        return acc
    
    return factorial_tail(n - 1, acc * n)

cProfile.run("factorial(1000)")

## Lab 09.03 - List Comprehension with Nested Lists
1. Given two strings, `s1` and `s2`, find all of the two-letter combinations possible from the strings using list comprehension

In [ ]:
def cross_strings(s1: str, s2: str) -> list[str]:
    return [a + b for a in s1 for b in s2]

print(cross_strings("String 1", "Python"))